In [7]:
!pip install yt-dlp opencv-python numpy zipfile36 scikit-image ultralytics transformers accelerate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.2/172.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.0/994.0 kB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [8]:
import os, cv2, yt_dlp, zipfile, numpy as np
from skimage.metrics import structural_similarity as ssim

VIDEO_FOLDER = "youtube_videos"
FRAME_FOLDER = "frames"
CLEANED_FRAME_FOLDER = "cleaned_frames"

os.makedirs(VIDEO_FOLDER, exist_ok=True)
os.makedirs(FRAME_FOLDER, exist_ok=True)
os.makedirs(CLEANED_FRAME_FOLDER, exist_ok=True)

def download_youtube_videos(video_urls):
    ydl_opts = {"format": "best", "outtmpl": f"{VIDEO_FOLDER}/%(id)s.%(ext)s"}
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download(video_urls)

def extract_frames(video_folder, frame_folder, frame_interval=30):
    for video_file in os.listdir(video_folder):
        if video_file.endswith(".mp4"):
            video_path = os.path.join(video_folder, video_file)
            video_id = video_file.split(".")[0]
            cap = cv2.VideoCapture(video_path)
            frame_count = 0
            success, frame = cap.read()
            while success:
                if frame_count % frame_interval == 0:
                    frame_filename = f"{frame_folder}/{video_id}_frame{frame_count}.jpg"
                    cv2.imwrite(frame_filename, frame)
                success, frame = cap.read()
                frame_count += 1
            cap.release()

def is_blurry(image, threshold=100):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    lap_var = cv2.Laplacian(gray, cv2.CV_64F).var()
    return lap_var < threshold

def are_images_similar(img1, img2, threshold=0.9):
    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    score, _ = ssim(gray1, gray2, full=True)
    return score > threshold

def clean_frames(frame_folder, cleaned_folder):
    prev_frame = None
    for frame_file in sorted(os.listdir(frame_folder)):
        frame_path = os.path.join(frame_folder, frame_file)
        image = cv2.imread(frame_path)
        if image is None or is_blurry(image):
            continue
        if prev_frame is not None and are_images_similar(prev_frame, image):
            continue
        prev_frame = image.copy()
        cv2.imwrite(os.path.join(cleaned_folder, frame_file), image)

video_urls = ["https://www.youtube.com/watch?v=M3Yo1z3qLgU"]
download_youtube_videos(video_urls)
extract_frames(VIDEO_FOLDER, FRAME_FOLDER, frame_interval=15)
clean_frames(FRAME_FOLDER, CLEANED_FRAME_FOLDER)


[youtube] Extracting URL: https://www.youtube.com/watch?v=M3Yo1z3qLgU
[youtube] M3Yo1z3qLgU: Downloading webpage
[youtube] M3Yo1z3qLgU: Downloading tv client config
[youtube] M3Yo1z3qLgU: Downloading player 9599b765-main
[youtube] M3Yo1z3qLgU: Downloading tv player API JSON
[youtube] M3Yo1z3qLgU: Downloading ios player API JSON
[youtube] M3Yo1z3qLgU: Downloading m3u8 information
[info] M3Yo1z3qLgU: Downloading 1 format(s): 18
[download] Destination: youtube_videos/M3Yo1z3qLgU.mp4
[download] 100% of   66.00MiB in 00:00:01 at 39.82MiB/s  


In [9]:
from ultralytics import YOLO
import json

model = YOLO("yolov8n.pt")

def detect_and_visualize_player(frame_folder, save_folder="tracked_frames"):
    os.makedirs(save_folder, exist_ok=True)
    tracking_data = {}

    for frame_file in sorted(os.listdir(frame_folder)):
        path = os.path.join(frame_folder, frame_file)
        image = cv2.imread(path)
        if image is None:
            continue

        result = model(image)[0]
        player_boxes = [box for box in result.boxes if int(box.cls) == 0]
        if not player_boxes:
            continue

        x1, y1, x2, y2 = player_boxes[0].xyxy[0].tolist()
        cx, cy = int((x1 + x2) / 2), int((y1 + y2) / 2)

        cv2.circle(image, (cx, cy), 10, (0, 255, 0), -1)
        cv2.imwrite(os.path.join(save_folder, frame_file), image)

        tracking_data[frame_file] = {"position": [cx, cy], "bbox": [x1, y1, x2, y2]}

    return tracking_data

tracking_data = detect_and_visualize_player(CLEANED_FRAME_FOLDER)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 71.0MB/s]



0: 384x640 14 persons, 1 traffic light, 1 tie, 1 clock, 363.5ms
Speed: 5.6ms preprocess, 363.5ms inference, 38.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 1 toothbrush, 151.4ms
Speed: 3.0ms preprocess, 151.4ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 142.8ms
Speed: 2.7ms preprocess, 142.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 1 tie, 159.2ms
Speed: 2.6ms preprocess, 159.2ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 boats, 1 umbrella, 141.7ms
Speed: 2.3ms preprocess, 141.7ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 139.6ms
Speed: 2.2ms preprocess, 139.6ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 140.8ms
Speed: 2.4ms preprocess, 140.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 persons, 143.3ms

In [17]:
import requests

# Together AI API Key
# API_KEY = "your_together_api_key_here"  # Replace with your actual Together API key

# Preferred model (Mixtral is powerful & free)
model = "meta-llama/Llama-3.3-70B-Instruct-Turbo-Free"

# Build the prompt from tracking data
def build_prompt(tracking_data):
    prompt = "Analyze the movement pattern of the player in these frames:\n\n"
    for frame, info in tracking_data.items():
        prompt += f"- {frame}: Position {info['position']}\n"
    prompt += "\nWho might this player be based on consistent appearances and what can you infer?"
    return prompt

# Generate prompt
llm_prompt = build_prompt(tracking_data)

# Truncate prompt if needed
short_prompt = llm_prompt[-2000:]

# Send request to Together AI
headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

data = {
    "model": model,
    "prompt": short_prompt,
    "max_tokens": 300,
    "temperature": 0.7,
    "top_p": 0.9,
}

response = requests.post("https://api.together.xyz/v1/completions", json=data, headers=headers)

# Print the AI's analysis
print("\n--- AI Analysis ---\n")
print(response.json()["choices"][0]["text"].strip())



--- AI Analysis ---

The player might be a soccer player, likely a midfielder, given the consistent appearances near the center of the field. The player seems to be involved in various plays, moving around the field to support both defense and offense. The positions suggest a high level of activity and involvement in the game, indicating a key player in the team's strategy. 

Please note that without more information or context about the game, team, or specific players, it's challenging to provide a more precise identification or detailed analysis of the player's role or performance. 

However, we can make some general observations and inferences based on the data provided:

1. **Mobility and Involvement**: The player's positions across different frames indicate a high level of mobility and involvement in the game. This suggests that the player is likely a key figure in the team, possibly playing a central role such as a midfielder.

2. **Strategic Importance**: The varied positions, 

In [16]:
import gradio as gr
import requests

# Uses tracking_data from: tracking_data = detect_and_visualize_player(CLEANED_FRAME_FOLDER)

# Smart prompt builder using real YOLO output
def build_smart_prompt(user_question, tracking_data):
    intro = (
        "You are an AI soccer analyst. You are analyzing player movement across multiple video frames from a soccer match.\n"
        "The following are the tracked positions of one player across time:\n\n"
    )
    position_list = ""
    for frame, info in tracking_data.items():
        position_list += f"- {frame}: Position {info['position']}\n"
    closing = f"\nNow, based on these positions, answer the following question:\n{user_question}"
    return intro + position_list + closing

# Send request to Together AI
def analyze_player(user_question):
    # API_KEY = "your_together_api_key_here"  # Replace this
    MODEL = "mistralai/Mixtral-8x7B-Instruct-v0.1"

    prompt = build_smart_prompt(user_question, tracking_data)

    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    data = {
        "model": MODEL,
        "prompt": prompt[-2000:],  # Trim if needed
        "max_tokens": 300,
        "temperature": 0.7,
        "top_p": 0.9,
    }

    try:
        res = requests.post("https://api.together.xyz/v1/completions", json=data, headers=headers)
        return res.json()["choices"][0]["text"].strip()
    except Exception as e:
        return f"❌ Error: {e}"

# 🖼️ Gradio UI (simple and clean)
gr.Interface(
    fn=analyze_player,
    inputs=gr.Textbox(
        label="Ask about the player",
        value="Who might this player be based on consistent appearances and what can you infer?",
        lines=3
    ),
    outputs=gr.Textbox(label="AI's Soccer Analysis", lines=12),
    title="⚽ AI Soccer game Analyzer",
    description="Ask an AI about a soccer player's movement."
).launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7bd6b22d36cf619728.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
